# **Step 1° - Convolution of hyperspectral data**

In [ ]:
import os
import pandas as pd

# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")

# Reading the CSV file
DB_raw = pd.read_csv("DB_validation_raw_clustered.csv", encoding='ISO-8859-1')

hyper_data = DB_raw

# Select the ID column number and the numbers where the hyperspectral bands begin and end
hyper_data = hyper_data.iloc[:, [0] + list(range(16, 2167))]

In [ ]:
print(hyper_data)

In [ ]:
##################################################################
#  At this stage of the script, data from hyperspectral bands    #
#  to multispectral bands is convolved.                          #
#  Pay attention to the script comments at this stage            #
##################################################################


# Imports
# Se estiver executando o script na máquina pessoal, insira o diretório da linguagem R instalada na máquina
os.environ['R_HOME'] = 'C:/Program Files/R/R-4.3.2'
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Saving Dataframes to CSV

os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
hyper_data.to_csv("hyper_data.csv", index=False)

# R Script
r_script = """

# Reading CSV files

# Ajuste do limite de memória no início do script
#memory.limit(size=20000000)

setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
hyper_data <- read.csv("hyper_data.csv", header = TRUE, sep = ",")

rownames(hyper_data) <- hyper_data[, 1]
hyper_data1 <- hyper_data
hyper_data1 <- hyper_data1[, -1]
ID_Unique <- hyper_data[, 1]
colnames(hyper_data1) <- seq(from = 350, to = 2500, by = 1)


##############################################################################################
## R: Directory path to the required hsdar package libraries##                               #
#                                                                                            #
.libPaths("G:/OneDrive/Documentos/Doutorado/protocol_database/libraires")                    #
#                                                                                            #
##############################################################################################

# Load the hsdar package
library(hsdar)


# Create a matrix of the data
hyper_data.matrix <- as.matrix(hyper_data1)

# Wavelengths for hyperspectral data
wave.hyper_data <- seq(from = 350, to = 2500, by = 1)

# Create speclib of the data
hyper_data.speclib <- speclib(hyper_data.matrix, wave.hyper_data)

# Get characteristics of the sensor
get.sensor.characteristics("Sentinel2a", response_function = TRUE)


############################################################################################
#The following command will obtain the channel wavelength of deployed
#satellite sensors (multispectral).
#Below is a list of sensor names available for operation.
#Copy and paste the name into the command:

#"Landsat4"
#"Landsat5"
#"Landsat7"
#"Landsat8"
#"Sentinel2a"
#"Sentinel2b"
#"RapidEye
#"WorldView2-8"
#"Quickbird"
#"WorldView2-4"

# Perform spectral resampling
multi_data.sentinel2.1 <- spectralResampling(hyper_data.speclib, "Sentinel2a",
                                            response_function = TRUE)

############################################################################################

# Plot a graph
plot(multi_data.sentinel2.1[1])

# Summary statistics
summary(multi_data.sentinel2.1)


# Round the values before transforming into a dataframe
resampled_spectra <- as.data.frame(multi_data.sentinel2.1)

#resampled_spectra <- round(as.data.frame(multi_data.sentinel2.1))

# Add IDs back
resampled_spectra <- cbind(ID_Unique, resampled_spectra)

# Write the dataframe to a CSV file
setwd("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
write.csv(resampled_spectra, "resampled_spectra.csv", row.names = FALSE)

"""
# Executing the R script
robjects.r(r_script)

In [ ]:
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/validation_CS/protocol_DB_validation")
# Reading the CSV file

resampled_spectra = pd.read_csv("resampled_spectra.csv")

print(resampled_spectra)

### **Data handling for Landsat 8 spectral resolution**

In [ ]:
# Landsat 8 spectral resolution:

# B1.........Costal...........430-450 (nm)
# B2.........Blue.............450-510 (nm)
# B3.........Green............530-590 (nm)
# B4.........Red..............640-670 (nm)
# B5.........NIR..............850-880 (nm)
# B9.........Cirrus.........1360-1380 (nm)
# B6.........SWIR1..........1570-1650 (nm)
# B7.........SWIR2..........2110-2290 (nm)


# Create a dictionary of correspondence between original and new names
corresponding_name = {
    "ID_Unique": "ID_Unique",
    "Costal": "Costal_Landsat8_resampled",
    "Blue": "Blue_Landsat8_resampled",
    "Green": "Green_Landsat8_resampled",
    "Red": "Red_Landsat8_resampled",
    "NIR": "NIR_Landsat8_resampled",
    "Cirrus": "Cirrus_Landsat8_resampled",
    "SWIR1": "SWIR1_Landsat8_resampled",
    "SWIR2": "SWIR2_Landsat8_resampled"
}

# Rename the columns of the resampled_spectra dataframe
resampled_spectra.columns = [corresponding_name[col] for col in resampled_spectra.columns]

# Merge the dataframes
DB_raw_resampled_spectra = pd.merge(DB_raw, resampled_spectra, on="ID_Unique")

In [ ]:
print(DB_raw_resampled_spectra)

In [ ]:
# Saving Dataframes to CSV
DB_raw_resampled_spectra.to_csv("DB_raw_resampled_spectra.csv", index=False)

### **Data handling for Sentinel-2A spectral resolution**

In [ ]:
# Sentinel-2A spectral resolution:
# B1.........Aerossol.........430-450 (nm)
# B2.........Blue.............458-523 (nm)
# B3.........Green............543-578 (nm)
# B4.........Red..............650-680 (nm)
# B5.........Red Edge 1.......698-713 (nm)
# B6.........Red Edge 2.......733-748 (nm)
# B7.........Red Edge 3.......773-793 (nm)
# B8.........NIR..............785-899 (nm)
# B8A........Red Edge 4.......855-875 (nm)
# B9.........Water Vapor......930-950 (nm)
# B10........Cirrus...........1360-1390 (nm)
# B11........SWIR 1...........1565-1655 (nm)
# B12........SWIR 2...........2100-2280 (nm)

# Create a dictionary of correspondence between original and new names
corresponding_name = {
    "ID_Unique": "ID_Unique",
    "SR_AV_B1": "Aerosol_Sentinel2A_resampled",
    "SR_AV_B2": "Blue_Sentinel2A_resampled",
    "SR_AV_B3": "Green_Sentinel2A_resampled",
    "SR_AV_B4": "Red_Sentinel2A_resampled",
    "SR_AV_B5": "Red_Edge_1_Sentinel2A_resampled",
    "SR_AV_B6": "Red_Edge_2_Sentinel2A_resampled",
    "SR_AV_B7": "Red_Edge_3_Sentinel2A_resampled",
    "SR_AV_B8": "NIR_Sentinel2A_resampled",
    "SR_AV_B8A": "Red_Edge_4_Sentinel2A_resampled",
    "SR_AV_B9": "Water_Vapor_Sentinel2A_resampled",
    "SR_AV_B10": "Cirrus_Sentinel2A_resampled",
    "SR_AV_B11": "SWIR_1_Sentinel2A_resampled",
    "SR_AV_B12": "SWIR_2_Sentinel2A_resampled"
}

# Rename the columns of the resampled_spectra dataframe
resampled_spectra.columns = [corresponding_name[col] for col in resampled_spectra.columns]

# Merge the dataframes
DB_raw_resampled_spectra = pd.merge(DB_raw, resampled_spectra, on="ID_Unique")

In [ ]:
print(DB_raw_resampled_spectra)

In [ ]:
# Saving Dataframes to CSV
DB_raw_resampled_spectra.to_csv("DB_validation_raw_resampled_spectra.csv", index=False)

# **Step 2° - Filter by equations**

## **Step 2.2 - Filter equation tendency**

The tendency equation is a rule that expresses the spectral signature of the soil. The spectral bands Blue, Green, Red, and NIR must necessarily follow the order of increasing values. This order is: Blue < Green < Red < NIR.

In [ ]:
import os
import pandas as pd

# Setting the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania")

# Reading the CSV file
DB_raw = pd.read_csv("DB_Worldspecs_multi_raw_Oceania_8.csv", sep=',', encoding='ISO-8859-1')

# Exibindo o DataFrame incial
print(DB_raw)

In [ ]:
import pandas as pd

# Creating the new DataFrame
results_equation_tendency = pd.DataFrame()

# Adding the 'ID_Unique' column to the new DataFrame
results_equation_tendency['ID_Unique'] = DB_raw['ID_Unique']

############################################################################
# Change the names of the spectral bands, Blue, Green, Red, and NIR, if
# the names of these respective bands in the columns of your dataset
# are different.

# Adding the 'outlier_tendency' column based on the condition
condition = (
    (DB_raw['Blue_SySI_Sentinel'] < DB_raw['Green_SySI_Sentinel']) &
    (DB_raw['Green_SySI_Sentinel'] < DB_raw['Red_SySI_Sentinel']) &
    (DB_raw['Red_SySI_Sentinel'] < DB_raw['NIR_SySI_Sentinel'])
)
############################################################################

results_equation_tendency['outlier_tendency'] = ~condition

# Displaying the new DataFrame
#print(results_equation_tendency)

# Adding the 'outlier_tendency' column to the original DataFrame using merge
DB_tendency = pd.merge(DB_raw, results_equation_tendency[['ID_Unique', 'outlier_tendency']], on='ID_Unique')

# Storing the 'outlier_tendency' column in a temporary variable
outlier_tendency_col = DB_tendency['outlier_tendency']

# Removing the 'outlier_tendency' column from the DataFrame
DB_tendency = DB_tendency.drop('outlier_tendency', axis=1)

# Inserting the 'outlier_tendency' column at the desired position (position 2)
DB_tendency.insert(2, 'outlier_tendency', outlier_tendency_col)

# DataFrame for True values in 'outlier_tendency'
outliers_samples_tendency = DB_tendency[DB_tendency['outlier_tendency'] == True]

# DataFrame for False values in 'outlier_tendency'
DB_filter_tendency = DB_tendency[DB_tendency['outlier_tendency'] == False]

# Displaying the final DataFrame
print(DB_filter_tendency)


In [ ]:
# Checking the number of samples in the initial dataframe,
# outliers tendency dataframe, and filtered dataframe

# Print the number of samples in DB_filter_lignin
print("Number of samples in DB_raw:", DB_raw.shape[0])

# Print the number of samples in outliers_samples_tendency
print("Number of samples in outliers_samples_tendency:", outliers_samples_tendency.shape[0])

# Print the number of samples in DB_filter_tendency
print("Number of samples in DB_filter_tendency:", DB_filter_tendency.shape[0])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_spectra(data, plot_name: str = 'spectra_plot'):
    # Dicionário para mapear nomes de colunas para rótulos mais curtos
    label_map = {
        'Blue_SySI_Sentinel': 'Blue (458-523 nm)',
        'Green_SySI_Sentinel': 'Green (543-578 nm)',
        'Red_SySI_Sentinel': 'Red (650-680 nm)',
        'Red_Edge_1_10m_SySI_Sentinel': 'Red Edge 1 (698-713 nm)',
        'Red_Edge_2_10m_SySI_Sentinel': 'Red Edge 2 (733-748 nm)',
        'Red_Edge_3_10m_SySI_Sentinel': 'Red Edge 3 (773-793 nm)',
        'NIR_SySI_Sentinel': 'NIR (785-899 nm)',
        'Red_Edge_4_10m_SySI_Sentinel': 'Red Edge 4 (855-875 nm)',
        'SWIR_1_10m_SySI_Sentinel': 'SWIR 1 (1565-1655 nm)',
        'SWIR_2_10m_SySI_Sentinel': 'SWIR 2 (2100-2280 nm)'
    }

    # Especificar as colunas espectrais
    spectral_columns = list(label_map.keys())

    # Filtrar colunas que existem no DataFrame
    existing_columns = [col for col in spectral_columns if col in data.columns]
    # Obter rótulos curtos
    short_labels = [label_map[col] for col in existing_columns]

    plt.figure(figsize=(12, 7))

    # Iterando sobre cada linha nos dados
    for index, row in data.iterrows():
        # Plotar cada linha com base nas colunas espectrais
        plt.plot(short_labels, row[existing_columns], label=f'Outlier {index}')

    plt.xlabel('Spectral Band')
    plt.ylabel('Reflectance Factor')
    plt.title(f'Spectral Signatures of {plot_name}')

    # Rotacionar os labels do eixo x para evitar sobreposição
    plt.xticks(rotation=45)

    # Definir limites para o eixo y
    plt.ylim(0, 0.8)  # Ajuste conforme necessário

    plt.grid(False)

    # Salvar o gráfico com alta resolução
    output_path = "G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering/DB_Raw.png"
    dpi_value = 1200
    plt.savefig(output_path, dpi=dpi_value, bbox_inches='tight')

    plt.show()


In [ ]:
plot_spectra(DB_filter_tendency, 'DB Raw')

In [ ]:
# Save the DB_filter_step_2 DataFrame as a CSV file
# Setting the working directory
os.chdir("G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/SySI_data_filtering")

DB_filter_tendency.to_csv("DB_filter_step_2_SySI_data.csv", index=False)
outliers_samples_tendency.to_csv("outliers_samples_tendency.csv", index=False)

# **Step 3° - Filter by PCA and distance Mahalanobis**

In [ ]:
# Packages required
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import os

In [ ]:
# Setting the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania")

# Importing soil spectra
DB_filter_step_2 = pd.read_csv("DB_wosis_202312_clay_orgc_5_Oceania.csv", sep=',', encoding='ISO-8859-1')
DB_filter_step_2 = DB_filter_step_2.set_index('ID_Unique') #Assigning row names from ID column

print(DB_filter_step_2)

In [ ]:
### spectra selection ###

# Selecting specific columns where the spectrum begins and ends

# Specify the range of spectral columns
wavelength_min = "Blue_SySI_Sentinel" # adjust the min spectral wavelength
wavelength_max = "SWIR_2_10m_SySI_Sentinel" # adjust the max spectral wavelength

# Select the spectral columns
data = DB_filter_step_2.loc[:, wavelength_min:wavelength_max] # soil spectra (original)

print(data)

In [ ]:
### standardized data ###

data_selected = data.apply(pd.to_numeric, errors='coerce')
data_selected.fillna(data_selected.min(), inplace=True)

# Standardize the data
scaler = MinMaxScaler()
data_standardized = pd.DataFrame(scaler.fit_transform(data), index=data.index)

# Convert the standardized data back to a DataFrame
data_standardized_df = pd.DataFrame(data_standardized)

In [ ]:
print(data_standardized)

In [ ]:
#IDENTIY OUTLIERS FUNCTION
def identify_outliers(x, n_pc_comp=0):
    # Standardize the data
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    # Handle NaN values (replace with mean of each column)
    x_scaled = np.nan_to_num(x_scaled, nan=np.nanmean(x_scaled, axis=0))

    # Perform PCA
    pca = PCA()
    pr_scores = pca.fit_transform(x_scaled)
    explained_variance = pca.explained_variance_ratio_
    cpr = np.cumsum(explained_variance)

    # Determine number of principal components
    if n_pc_comp == 0:
        n_pc_comp = np.min(np.where(cpr > 0.999)[0] + 1)
    elif 0 < n_pc_comp < 1:
        n_pc_comp = np.min(np.where(cpr > n_pc_comp)[0] + 1)

    # Calculate mean and covariance
    mean_pcaA = np.mean(pr_scores[:, :n_pc_comp], axis=0)
    cov_pcaA = np.cov(pr_scores[:, :n_pc_comp], rowvar=False)

    # Calculate Mahalanobis distance
    chiMat = np.zeros((pr_scores.shape[0], 3))
    chiMat[:, 0] = np.array([np.dot(np.dot((pr_scores[i, :n_pc_comp] - mean_pcaA), np.linalg.inv(cov_pcaA)), (pr_scores[i, :n_pc_comp] - mean_pcaA).T) for i in range(pr_scores.shape[0])])

    # Fit Chi-Squared distribution
    chiMat[:, 1] = chi2.cdf(chiMat[:, 0], df=n_pc_comp)

    # Determine critical value
    if n_pc_comp <= 10:
        pcrit = 1 - ((0.24 - 0.003 * n_pc_comp) / np.sqrt(pr_scores.shape[0]))
    else:
        pcrit = 1 - ((0.25 - 0.0018 * n_pc_comp) / np.sqrt(pr_scores.shape[0]))
    pcrit = 0.999
    #pcrit = 0.995

    # Identify outliers
    chiMat[:, 2] = np.where(chiMat[:, 1] >= pcrit, 0, 1)

    # Return the unique IDs of the outliers
    outliers_indices = np.where(chiMat[:, 2] == 0)[0]
    outliers_ids = x.index[outliers_indices]

    return outliers_ids


In [ ]:
# Calling the function and passing the original DataFrame
outliers_ids = identify_outliers(data, n_pc_comp=0.995)

# Exibir os IDs únicos dos outliers
print(outliers_ids)

In [ ]:
filtered_data_step_3 = data.drop(index=outliers_ids)
outliers_data_step_3 = data.loc[outliers_ids]

print(outliers_data_step_3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import PCA
import os

def plot_spectra(data, plot_name: str = 'spectra_plot', save_path=None, dpi: int = 1000):
    # Dicionário para mapear nomes de colunas para rótulos mais curtos
    label_map = {
        'Blue_SySI_Sentinel': 'Blue (458-523 nm)',
        'Green_SySI_Sentinel': 'Green (543-578 nm)',
        'Red_SySI_Sentinel': 'Red (650-680 nm)',
        'Red_Edge_1_10m_SySI_Sentinel': 'Red Edge 1 (698-713 nm)',
        'Red_Edge_2_10m_SySI_Sentinel': 'Red Edge 2 (733-748 nm)',
        'Red_Edge_3_10m_SySI_Sentinel': 'Red Edge 3 (773-793 nm)',
        'NIR_SySI_Sentinel': 'NIR (785-899 nm)',
        'Red_Edge_4_10m_SySI_Sentinel': 'Red Edge 4 (855-875 nm)',
        'SWIR_1_10m_SySI_Sentinel': 'SWIR 1 (1565-1655 nm)',
        'SWIR_2_10m_SySI_Sentinel': 'SWIR 2 (2100-2280 nm)'
    }
    # Especificar as colunas espectrais
    spectral_columns = list(label_map.keys())
    # Filtrar colunas que existem no DataFrame
    existing_columns = [col for col in spectral_columns if col in data.columns]
    # Obter rótulos curtos
    short_labels = [label_map[col] for col in existing_columns]
    plt.figure(figsize=(12, 6))

    # Iterando sobre cada linha nos dados
    for index, row in data.iterrows():
        # Plotar cada linha com base nas colunas espectrais
        plt.plot(short_labels, row[existing_columns], label=f'Outlier {index}')

    plt.xlabel('Spectral Band')
    plt.ylabel('Reflectance Factor')
    plt.title(f'Spectral Signatures of {plot_name}')

    # Rotacionar os labels do eixo x para evitar sobreposição
    plt.xticks(rotation=45)
    # Definir limites para o eixo y
    plt.ylim(0, 1)  # Ajuste conforme necessário
    plt.grid(False)
    plt.savefig(f'{plot_name}.png', dpi=dpi, bbox_inches='tight')
    plt.show()


def plot_3d_pca_with_outliers(data_standardized, outliers_ids, save_path=None, dpi: int = 1000):
    """
    Plots a 3D PCA plot highlighting outliers.

    Parameters:
    - data_standardized: The standardized data (samples x features).
    - outliers_ids: Indices of the outliers in the dataset.
    - save_path: (Optional) Path to save the plot. If None, the plot is not saved.
    """
    # Handle NaN values before applying PCA
    data_standardized = data_standardized.fillna(data_standardized.mean()) # Fill NaN with column means

    # Perform PCA
    pca = PCA(n_components=3)
    pr_scores = pd.DataFrame(pca.fit_transform(data_standardized), index=data_standardized.index)

    # Extracting the outliers and non-outliers
    outliers = pr_scores.loc[outliers_ids]
    non_outliers = pr_scores.drop(index=outliers_ids)

    # Convert outliers and non_outliers to numpy arrays for plotting
    outliers_array = outliers.to_numpy()
    non_outliers_array = non_outliers.to_numpy()

    # Create 3D plot
    fig = plt.figure(figsize=(15, 13))
    ax = fig.add_subplot(111, projection='3d')

    # Plot outliers
    ax.scatter(outliers_array[:, 0], outliers_array[:, 1], outliers_array[:, 2], c='red', marker='x', s=75, label="Outliers")

    # Plot non-outliers
    ax.scatter(non_outliers_array[:, 0], non_outliers_array[:, 1], non_outliers_array[:, 2], c='gray', marker='o', alpha=0.6,
                                                                              edgecolors='black', linewidths=0.45, s=30, label="Non-outliers")

    # Label axes with explained variance ratios
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0] * 100:.2f}%)', fontweight='bold', fontsize=28, labelpad=20)
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1] * 100:.2f}%)', fontweight='bold', fontsize=28, labelpad=28)
    ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2] * 100:.2f}%)', fontweight='bold', fontsize=28, labelpad=22)

    # Increase the font size of the tick labels
    ax.tick_params(axis='both', which='major', labelsize=26, pad=2.8)  # Adjusts x and y axis
    ax.tick_params(axis='z', which='major', labelsize=26, pad=7.5)     # Adjusts z axis

    # Add legend
    ax.legend()

    # Adjust layout to prevent label cutoff
    plt.tight_layout()

    # Set the plot title
    plt.title('3D PCA plot with Outliers Oceania', fontweight='bold', fontsize=20, y=1.05)

    # Save the figure if a path is provided
    if save_path:
        plt.savefig(save_path, dpi=1200)

    # Show plot
    plt.show()


In [ ]:
# Plots Visualization

# Setting the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_3")

plot_spectra(filtered_data_step_3, 'filtered data step 3 Oceania')
plot_spectra(outliers_data_step_3, 'outliers Oceania')
plot_spectra(data, 'all data Oceania')
plot_3d_pca_with_outliers(data, outliers_ids, save_path='3d_pca_outliers_Oceania_multi')

In [ ]:
outliers_data_step_3 = outliers_data_step_3.reset_index()

IDs_to_filter = outliers_data_step_3['ID_Unique'].tolist()

DB_filter_step_3 = DB_filter_step_2.reset_index()

# Add a new column "outliers_step_3" filled with False
DB_filter_step_3.insert(1, "outliers_step_3", False)

# Update the values to True where ID_Unique is in the list IDs_to_filter
DB_filter_step_3.loc[DB_filter_step_3['ID_Unique'].isin(IDs_to_filter), 'outliers_step_3'] = True

# Create a new DataFrame with only the rows where outliers_step_3 is True
outliers_samples_step_3 = DB_filter_step_3[DB_filter_step_3['outliers_step_3'] == True].copy()

# Remove the samples where outliers_step_3 is True
DB_filter_step_3 = DB_filter_step_3[DB_filter_step_3['outliers_step_3'] == False].copy()

print(outliers_samples_step_3)

In [ ]:
# Checking the number of samples in the initial dataframe,
# outliers_samples_step_3 dataframe, and filtered dataframe

# Print the number of samples in DB_filter_step_2
print("Number of samples in DB_filter_step_2:", DB_filter_step_2.shape[0])

# Print the number of samples in outliers_step_3
print("Number of samples in outliers_samples_step_3:", outliers_samples_step_3.shape[0])

# Print the number of samples in DB_filter_step_3
print("Number of samples in DB_filter_step_3:", DB_filter_step_3.shape[0])

In [ ]:
# Setting the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_3")

# Save the outliers_samples_3 DataFrame as a CSV file
outliers_samples_step_3.to_csv("outliers_samples_step_3.csv", index=False)

# Save the DB_filter_3 DataFrame as a CSV file
DB_filter_step_3.to_csv("DB_multi_filter_step_3_Oceania_10.csv", index=False)

## **Visual analysis of the outiliers determined by Step 3°**

This sub-step is not mandatory for completing the entire protocol, but we strongly recommend that the spectral signatures of samples identified as outliers be visually analyzed. This analysis allows the pedometrist to understand the nature of the outlier spectrum and why the data might be inconsistent or incorrect. Examples include:

- Miscalibration of the Vis-NIR-SWIR sensor, resulting in noise in the spectral signature.
- Contamination of the soil sample with other materials.
- Improper preparation of the soil sample.
- Collection of the spectral data from a target other than the soil sample.

By understanding the nature of the error, the pedometrist can intervene at the source, preventing similar errors from occurring in the future.
However, it is important to note that the spectral data may be correct. The combination of PCA and Mahalanobis distance may flag an outlier that actually corresponds to a soil with distinct spectral behavior, which is not well-represented in the dataset. Therefore, if you identify a sample whose spectral behavior matches all the correct spectral characteristics of a soil in the "outliers_samples_step_3" dataset, we recommend including this sample's data in the "DB_filter_step_3" dataset.

The following interactive plots of the hyperspectral data from soil samples will be generated for visual analysis.

In [ ]:
# Packages required
import os
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
print(outliers_samples_step_3)

In [ ]:
# Define the indices of the columns you want to select
col_index_min = 109  # Index of the first spectral column
col_index_max = 2189  # Index of the last spectral column

# Select the ID column by name and the spectral columns by indices
selected_columns = [outliers_samples_step_3.columns.get_loc('ID_Unique')] + list(range(col_index_min, col_index_max + 1))

# Create a new DataFrame with the desired columns
DB_graphic = outliers_samples_step_3.iloc[:, selected_columns].copy()

print(DB_graphic)

In [ ]:
# Split the dataframe into smaller dataframes based on rows, in this case 50 rows
dfs = np.array_split(DB_graphic, np.ceil(len(DB_graphic) / 100))

# Create the line chart using Plotly Express for each smaller dataframe
for i, df in enumerate(dfs):
    # Reshape the dataframe
    df_melt = df.melt(id_vars=df.columns[0], value_vars=df.columns[1:])

    fig = px.line(df_melt, x='variable', y='value', color=df.columns[0],
                  title=f'Original Spectra {i+1}', labels={'variable': 'Wavelength (nm)', 'value': 'Reflectance Factor'})

    # Customize the chart layout
    fig.update_layout(
        xaxis_title='Wavelength (nm)',
        yaxis_title='Reflectance Factor',
    )

    # Build the HTML file name based on the chart
    html_file_name = os.path.join('G:/OneDrive/Documentos/Doutorado/protocol_database/protocol_update/Step_3', f'Original_Spectra_Plotly_{i+1}.html')

    # Save the interactive chart to an HTML file
    fig.write_html(html_file_name)

    # Display the interactive chart
    fig.show()

# **Step 4° - Filter by Soil Line**

In [ ]:
# Packages required
import os
import statsmodels.api as sm
import numpy as np
import pandas as pd
from plotnine import *
from plotnine import ggsave
import scipy.stats as stats
from plotnine import ggplot

In [ ]:
### Soil line on convolved spectra ###

# Set the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_3")

# Read the CSV file
DB_filter_step_3 = pd.read_csv("DB_multi_filter_step_3_Oceania_10.csv")

# Assigning to a variable for clarity
DB = DB_filter_step_3

print(DB_filter_step_3)

In [ ]:
### Cutting line determination ###

# Specify the X and Y columns
column_x_conv = DB['Red_SySI_Sentinel']
column_y_conv = DB['NIR_SySI_Sentinel']

# Create a linear regression model
X = sm.add_constant(column_x_conv)  # Add the constant for the intercept term
model = sm.OLS(column_y_conv, X).fit()

# Calculate predicted values
predictions = model.predict(X)

# Calculate the standard deviation of residuals
residuals = model.resid
std_residuals = np.std(residuals)

# Confidence level (e.g., 98%)
confidence_level = 0.99

# Quantile of the t distribution
quantile_t = stats.t.ppf((1 + confidence_level) / 2, df=len(residuals) - 2)

# Calculate upper and lower confidence limits
upper_limit_NIR_resam = predictions + quantile_t * (std_residuals * 1.35)
lower_limit_NIR_resam = predictions - quantile_t * (std_residuals * 1.35)

# Regression coefficients
intercept, coef_x = model.params

# Standard error of coefficients
std_error = model.bse

# T-value and p-value of coefficients
t_value = model.tvalues
p_value = model.pvalues

# Coefficient of determination (R²)
r_squared_convolved = model.rsquared

# Fit a linear regression model for the upper limit
model_upper_limit = sm.OLS(upper_limit_NIR_resam, X).fit()

# Fit a linear regression model for the lower limit
model_lower_limit = sm.OLS(lower_limit_NIR_resam, X).fit()

# Coefficients of intercept and slope for the upper limit
coef_intercept_upper, coef_slope_upper = model_upper_limit.params

# Coefficients of intercept and slope for the lower limit
coef_intercept_lower, coef_slope_lower = model_lower_limit.params

# Display coefficients
print("Coefficients for Upper Limit:")
print("Intercept (β0):", round(coef_intercept_upper, 4))
print("Slope (β1):", round(coef_slope_upper, 4))

print("\nCoefficients for Lower Limit:")
print("Intercept (β0):", round(coef_intercept_lower, 4))
print("Slope (β1):", round(coef_slope_lower, 4))

In [ ]:
### plot the Soil line by convolved data ###

# Desired width and height of the plot
plot_width = 9
plot_height = 6

soil_line_plot = (
    ggplot(DB, aes(x='Red_SySI_Sentinel', y='NIR_SySI_Sentinel')) +
    theme_classic() +
    geom_point(shape='o', color= 'black', fill= "#E49247", alpha=1, size=1.5, stroke=0.2) + # Data points with black contour +  # Data points
    geom_smooth(method='lm', se=False, color='#000000', size=1.2) +  # Trend line
    geom_line(aes(y=upper_limit_NIR_resam), color='#E41A1C', size=1) +  # Upper confidence interval line
    geom_line(aes(y=lower_limit_NIR_resam), color='#E41A1C', size=1) +  # Lower confidence interval line

    annotate('text', x=0.11, y=0.58, label="β upper limit: {:.3f}".format(round(coef_intercept_upper, 3)),
             family="Arial", size=10)+
    annotate('text', x=0.11, y=0.54, label="β lower limit: {:.3f}".format(round(coef_intercept_lower, 3)),
             family="Arial", size=10)+
    annotate('text', x=0.11, y=0.50, label="R² model: {:.3f}".format(round(r_squared_convolved, 3)),
             family="Arial", size=10)+
    annotate('text', x=0.11, y=0.46, label="α: {:.3f}".format(round(coef_x, 3)),
             family="Arial", size=10)+

    labs(x="SySI Red band (650-680 nm)", y="SySI NIR band (785-899 nm)") +
    expand_limits(x=0, y=0) +  # Force the plot to show the origin (0,0)
    scale_y_continuous(limits=[0, 0.6]) +
    scale_x_continuous(limits=[0, 0.6]) +
    theme(
        axis_title_x=element_text(margin={'t': 10}, family="Arial", size=12, face="bold"),
        axis_title_y=element_text(margin={'r': 10}, family="Arial", size=12, face="bold"),
        axis_text_y=element_text(family="Arial", size=10),
        axis_text_x=element_text(family="Arial", size=10)
    )
)


# Display the plot
print(soil_line_plot)

dpi_value = 1000

# Setting the working directory
output_path = "E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_4/soil_line_plot_Oceania.png"
soil_line_plot.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

soil_line_plot

In [ ]:
## After Soil Line analysis, filtering the samples in the database ###

# Create a subset of samples that are ABOVE the upper_limit_NIR line
samples_upper_limit = DB[DB['NIR_SySI_Sentinel'] > upper_limit_NIR_resam]

# Create a subset of samples that are BELOW the lower_limit_NIR line
samples_lower_limit = DB[DB['NIR_SySI_Sentinel'] < lower_limit_NIR_resam]

# Concatenate the dataframes samples_upper_limit and samples_lower_limit
outliers_samples = pd.concat([samples_upper_limit, samples_lower_limit])

# Merge the DataFrames based on the ID_Unique column and keep only the columns from the DB DataFrame
outliers_samples_step_4 = pd.merge(outliers_samples[['ID_Unique']], DB, on='ID_Unique', how='inner')

IDs_to_filter = outliers_samples['ID_Unique'].tolist()

DB_filter_step_4 = DB_filter_step_3

# Add a new column "outliers_step_4" filled with False
DB_filter_step_4.insert(1, "outliers_step_4", False)

# Update the values to True where ID_Unique is in the list IDs_to_filter
DB_filter_step_4.loc[DB_filter_step_4['ID_Unique'].isin(IDs_to_filter), 'outliers_step_4'] = True

# Create a new DataFrame with only the rows where outliers_step_4 is True
outliers_samples_step_4 = DB_filter_step_4[DB_filter_step_4['outliers_step_4'] == True].copy()

# Delete the samples where outliers_step_3 is True
DB_filter_step_4 = DB_filter_step_4[DB_filter_step_4['outliers_step_4'] == False].copy()

print(outliers_samples_step_4)

In [ ]:
# Checking the number of samples in the initial dataframe of the attribute that was analyzed,
# outliers outliers_samples_step_4, and DB_filter_step_4 dataframe

# Print the number of samples in DB_filter_step_3
print("Number of samples in DB_filter_step_3:", DB_filter_step_3.shape[0])

# Print the number of samples in outliers_samples_step_4
print("Number of samples in outliers_samples_step_4:", outliers_samples_step_4.shape[0])

# Print the number of samples in DB_filter_step_4
print("Number of samples in DB_filter_step_4:", DB_filter_step_4.shape[0])

In [ ]:
# Save the DB_filter_step_4 and outliers_samples_step_4 DataFrame as a CSV file
# Setting the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_4")

outliers_samples_step_4.to_csv("outliers_samples_step_4_SySI_data.csv", index=False)

DB_filter_step_4.to_csv("DB_multi_filter_step_4_Oceania_11.csv", index=False)

# **Step 5° - CIFOD Filter**

## URSSA module 1: Importing and pre-processing data

In [ ]:
# Packages required
import os
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import pairwise_distances
from scipy.interpolate import interp1d
from plotnine import ggplot

In [ ]:
# Set the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_4")

# Importing data (soil attributes + spectra)
Data_all = pd.read_csv("DB_multi_filter_step_4_Oceania_11.csv")

In [ ]:
print(Data_all)

In [ ]:
# Copia o DataFrame Data_all para Data
Data = Data_all.copy()

wavelength_min = "Blue_SySI_Sentinel" # adjust the min spectral wavelength
wavelength_max = "SWIR_2_10m_SySI_Sentinel" # adjust the max spectral wavelength

Spectra_original = Data.loc[:, wavelength_min:wavelength_max] # soil spectra (original)

# Seleciona a coluna ID_Unique
ID_Unique = Data['ID_Unique']

# Cria o DataFrame dataset_model contendo as colunas espectrais e a coluna ID_Unique
dataset_model = Spectra_original.copy()

# Insere a coluna ID_Unique como a primeira coluna
dataset_model.insert(0, 'ID_Unique', ID_Unique)

# Plotting of the first ten original spectra
plt.figure()
plt.plot(Spectra_original.columns, Spectra_original.iloc[:10].T.values)  # Use column names as is (without int conversion)
plt.title('Original Spectra')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Reflectance Factor')
plt.xticks(rotation=80)  # Rotate x labels if needed
plt.show()

In [ ]:
print(dataset_model)

## URSSA module 2: Unsupervised spectral clustering of samples


The script can be executed in two ways: hosted on a personal machine or through the Google API.

### ***API Google***

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.install_packages('randomUniformForest')
utils.install_packages('magrittr')
utils.install_packages('dplyr')
utils.install_packages('tidyverse')

In [ ]:
### MODULE 2: Unsupervised spectral clustering of samples ####

## 1) Unsupervised randomUniformForest classification,
## 2) Calculating spectral proximity from URF,
## 3) Multidimensional Scaling (MDS),
## 4) Clustering samples (K-means)


import rpy2.robjects as robjects
from rpy2.robjects.packages import importr


# Import the R packages
randomUniformForest = importr('randomUniformForest')
magrittr = importr('magrittr')
dplyr = importr('dplyr')
tidyverse = importr('tidyverse')


dataset_model.to_csv("dataset_model.csv", index=True)

# Define the R script
r_script = """

setwd("/content")
dataset_model <- read.csv("dataset_model.csv", header = TRUE, sep = ",", dec = ".", na.strings = "NA", check.names = FALSE)
dataset_model <- dataset_model %>% remove_rownames %>% column_to_rownames(var="ID_Unique")

wavelength_min <- "350"
wavelength_max <- "2500"

results <- unsupervised.randomUniformForest(object = dataset_model %>% select(wavelength_min:wavelength_max), # select spectra
                                              baseModel = "proximity", # c("proximity", "proximityThenDistance", "importanceThenDistance")
                                              endModel = "MDSkMeans", # c("MDSkMeans", "MDShClust", "MDS", "SpectralkMeans")
                                              endModelMetric = NULL,
                                              samplingMethod = "with bootstrap", # c("uniform univariate sampling","uniform multivariate sampling", "with bootstrap")
                                              MDSmetric = "metricMDS", # c("metricMDS", "nonMetricMDS")
                                              proximityMatrix = NULL,
                                              sparseProximities = FALSE,
                                              outliersFilter = FALSE,
                                              Xtest = NULL,
                                              predObject = NULL,
                                              metricDimension = 2,
                                              coordinates = c(1,2),
                                              bootstrapReplicates = 100,
                                              clusters = NULL,
                                              maxIters = NULL,
                                              importanceObject = NULL,
                                              maxInteractions = 2,
                                              reduceClusters = FALSE,
                                              maxClusters = 10,
                                              mapAndReduce = FALSE,
                                              OOB = FALSE,
                                              subset = NULL,
                                              seed = 2014,
                                              uthreads = "auto")

results_final_R <- data.frame("ID_Unique" = rownames(dataset_model),
                              "cluster" = results$unsupervisedModel$cluster,
                              dataset_model)

write.table(results_final_R, "results_final_R.csv", sep = ",", dec = ".",row.names = FALSE)

"""

# Execute the R script
robjects.r(r_script)


### ***Personal machine - local execution environment:***
  Attention, you must indicate the directory path where R is installed. The "randomUniformForest" library requires version 4.3.2. And within the script in R language it is necessary to indicate the path to the directory where the libraries are installed.

In [ ]:
### MODULE 2: Unsupervised spectral clustering of samples ####

## 1) Unsupervised randomUniformForest classification,
## 2) Calculating spectral proximity from URF,
## 3) Multidimensional Scaling (MDS),
## 4) Clustering samples (K-means)

############################################################################################
#  enter the directory of the R language installed on the machine
os.environ['R_HOME'] = 'C:/Program Files/R/R-4.3.2'
############################################################################################

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# Saving Dataframes to CSV

os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5")
dataset_model.to_csv("dataset_model.csv", index=True)

# Define the R script
r_script = """

############################################################################################
## R: Directory path to the required hsdar package libraries##                             #
#                                                                                          #
 .libPaths("E:/OneDrive/Documentos/Doutorado/protocol_database/libraires")                 #
#                                                                                          #
############################################################################################

# Load the doParallel library
library(doParallel)

# Configure parallel cluster (can use desired number of cores)
cl <- makeCluster(16)
registerDoParallel(cl)

library(dplyr)
library(tidyverse)
library(randomUniformForest)

############################################################################################

setwd("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5")

############################################################################################

dataset_model <- read.csv("dataset_model.csv", header = TRUE, sep = ",", dec = ".", na.strings = "NA", check.names = FALSE)
dataset_model <- dataset_model %>% remove_rownames %>% column_to_rownames(var="ID_Unique")

wavelength_min <- "Blue_SySI_Sentinel"
wavelength_max <- "SWIR_2_10m_SySI_Sentinel"

results <- unsupervised.randomUniformForest(object = dataset_model %>% select(wavelength_min:wavelength_max), # select spectra
                                              baseModel = "proximity", # c("proximity", "proximityThenDistance", "importanceThenDistance")
                                              endModel = "MDSkMeans", # c("MDSkMeans", "MDShClust", "MDS", "SpectralkMeans")
                                              endModelMetric = NULL,
                                              samplingMethod = "uniform multivariate sampling", # c("uniform univariate sampling","uniform multivariate sampling", "with bootstrap")
                                              MDSmetric = "metricMDS", # c("metricMDS", "nonMetricMDS")
                                              proximityMatrix = NULL,
                                              sparseProximities = FALSE,
                                              outliersFilter = FALSE,
                                              Xtest = NULL,
                                              predObject = NULL,
                                              metricDimension = 2,
                                              coordinates = c(1,2),
                                              bootstrapReplicates = 100,
                                              clusters = NULL,
                                              maxIters = 300,
                                              importanceObject = NULL,
                                              maxInteractions = 5,
                                              reduceClusters = FALSE,
                                              maxClusters = 7,
                                              mapAndReduce = FALSE,
                                              OOB = FALSE,
                                              subset = NULL,
                                              seed = 2014,
                                              uthreads = "auto")

results_final_R <- data.frame("ID_Unique" = rownames(dataset_model),
                              "cluster" = results$unsupervisedModel$cluster,
                              dataset_model)

write.table(results_final_R, "results_final_R.csv", sep = ",", dec = ".",row.names = FALSE)

"""

# Execute the R script
robjects.r(r_script)

In [ ]:
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5")
results_final = pd.read_csv("results_final_R.csv")

print(results_final)

In [ ]:
### manipulating results in dataframes ###

clusters = results_final

# Selecting only the 'ID_Unique' and 'cluster' columns from the clusters DataFrame
clusters = clusters[['ID_Unique', 'cluster']]

# Performing a merge between 'clusters' and 'Data_all' using 'ID_Unique' as the key
# This will join both DataFrames, keeping only the rows with matching 'ID_Unique'
Soil_data_clustered = pd.merge(clusters, Data_all, on='ID_Unique', how='inner')

# Saving the merged DataFrame 'Soil_data_clustered' to a CSV file
# The file is saved with a comma as the separator and a dot for decimals
Soil_data_clustered.to_csv('Soil_data_clustered.csv', sep=',', decimal='.', index=False)

##Identification of soil attribute outliers by Isolation Forest in each cluster

In [ ]:
# Packages required
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from numpy import where
from mpl_toolkits.mplot3d import Axes3D
from plotnine import *

In [ ]:
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5")
Soil_data_clustered = pd.read_csv("Soil_data_clustered.csv")

print(Soil_data_clustered)

In [ ]:

# Create an empty DataFrame called results
results = pd.DataFrame()

# Loop through clusters from 1 to 10
for i in range(1, 11):
    # Filter the DataFrame to select only the records where the "cluster" column is equal to i
    data_estratificado = Soil_data_clustered[Soil_data_clustered['cluster'] == i]

    # Find the columns starting with "Blue_SySI_Sentinel" and ending with "SWIR_2_10m_SySI_Sentinel"
    colunas_selecionadas = data_estratificado.loc[:, 'Blue_SySI_Sentinel':'SWIR_2_10m_SySI_Sentinel']

    # Calculate the mean and standard deviation of the selected columns
    medias = colunas_selecionadas.mean(numeric_only=True)
    desvio_padrao = colunas_selecionadas.std(numeric_only=True)

    # Create a temporary DataFrame with the results for the current cluster
    results_cluster = pd.DataFrame({
        'cluster': [i] * len(medias),
        'wavelength': medias.index,  # Keep the column names as strings
        'average_reflectance': medias.values,
        'sd': desvio_padrao.values
    })

    # Add the results of the current cluster to the main DataFrame
    results = pd.concat([results, results_cluster], ignore_index=True)

# Delete rows where 'average_reflectance' is NaN (null)
results = results.dropna(subset=['average_reflectance'])

# Dicionário para mapear nomes de colunas para rótulos mais curtos
label_map = {
    'Blue_SySI_Sentinel': 'Blue (458-523 nm)',
    'Green_SySI_Sentinel': 'Green (543-578 nm)',
    'Red_SySI_Sentinel': 'Red (650-680 nm)',
    'Red_Edge_1_10m_SySI_Sentinel': 'Red Edge 1 (698-713 nm)',
    'Red_Edge_2_10m_SySI_Sentinel': 'Red Edge 2 (733-748 nm)',
    'Red_Edge_3_10m_SySI_Sentinel': 'Red Edge 3 (773-793 nm)',
    'NIR_SySI_Sentinel': 'NIR (785-899 nm)',
    'Red_Edge_4_10m_SySI_Sentinel': 'Red Edge 4 (855-875 nm)',
    'SWIR_1_10m_SySI_Sentinel': 'SWIR 1 (1565-1655 nm)',
    'SWIR_2_10m_SySI_Sentinel': 'SWIR 2 (2100-2280 nm)'
}

# Aplicar o mapeamento de nomes
results['wavelength'] = results['wavelength'].replace(label_map)

# Definir a ordem correta dos rótulos espectrais
ordered_labels = list(label_map.values())

# Converter para categórico com ordem específica
results['wavelength'] = pd.Categorical(results['wavelength'], categories=ordered_labels, ordered=True)

# Create the color palette based on the number of clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3","#FF760A",
           "#FFD700","#0AB2D1", "#F781BF", "#999999", "#66C2A5", "#8C564B",
           "#1F78B4", "#B2DF8A", "#FB9A99", "#FDBF6F", "#CAB2D6", "#6A3D9A"]

# Desired width and height of the plot
plot_width = 14
plot_height = 7

spectral_behavior = (ggplot(results, aes(x='wavelength',
                                         y='average_reflectance',
                                         ymin='average_reflectance - sd',
                                         ymax='average_reflectance + sd',
                                         group='cluster')) +

  geom_line(aes(color='factor(cluster)'), size=0.85) +
  geom_ribbon(aes(fill='factor(cluster)'), alpha=0.2) +
  scale_y_continuous(limits=[0, 0.6]) +
  scale_color_manual(values=palette) +  # Manually set the colors
  scale_fill_manual(values=palette) +  # Manually set the fill colors
  labs(title='Spectral behavior of clusters Oceania',
       x='Wavelength (nm)',
       y='Reflectance factor',
       color='Cluster',   # Name the color legend
       fill='Cluster') +  # Name the fill legend
  theme_minimal() +
  theme(figure_size=(plot_width, plot_height),
        plot_title=element_text(size=15, face="bold"),
        axis_title_x=element_text(size=12, face="bold"),
        axis_title_y=element_text(size=12, face="bold"),
        axis_text_x=element_text(size=10, angle=45, hjust=1),  # Rotate x-axis labels
        legend_text=element_text(size=10),
        legend_title=element_text(size=12, face="bold"),
        plot_background=element_rect(fill="white")))

print(spectral_behavior)

dpi_value = 1200

# Setting the working directory
output_path = "E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5/SySI_spectral_behavior_of_clusters_Oceania.png"
spectral_behavior.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

#Show spectral_behavior
spectral_behavior

In [ ]:
# Data preparation
Data = Soil_data_clustered.dropna(subset=['C_gkg'])
Data = Data.set_index('ID_Unique')

# Select the cluster column and the variable of interest
Observation = Data[["cluster", "C_gkg"]]

# Filter null data in the "Clay_gkg" column
Observation = Observation[Observation['C_gkg'].notnull()]

wavelength_min = "Blue_SySI_Sentinel" # adjust the min spectral wavelength
wavelength_max = "SWIR_2_10m_SySI_Sentinel" # adjust the max spectral wavelength

Spectra_original = Data.loc[:, wavelength_min:wavelength_max] # soil spectra (original)

# Merge DataFrames to have all the information in one place
Observation_final = pd.merge(Observation, Spectra_original, left_index=True, right_index=True, how='left')

print(Observation_final)

In [ ]:
print(Observation_final)

In [ ]:
############ Isolation Forest ############

#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

# Load the dataset
data = Observation_final

# Remove the 'cluster' column to keep only spectral features
features = data.drop(columns=['cluster'])

# Initialize dictionaries to store results

all_anomaly_dict = {}
out_anomalies_dict = {}
plot_data_dict = {}

# Iterate over each unique cluster
for idx, cluster_label in enumerate(sorted(data['cluster'].unique())):

    # Filter data for the current cluster
    cluster_data = data[data['cluster'] == cluster_label]

    # Remove rows with missing values and keep only spectral features
    # Use features.columns to get the column names and select only those columns
    data_filtered = cluster_data[features.columns].dropna()

    # Normalize the data using StandardScaler
    scaler = StandardScaler()
    data_filtered_scaled = scaler.fit_transform(data_filtered)

    # Apply Isolation Forest for anomaly detection
    iforest = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)
    predictions = iforest.fit_predict(data_filtered_scaled)

    # Get the anomaly score
    anomaly_scores = iforest.decision_function(data_filtered_scaled)
    data_filtered['anomaly_score'] = anomaly_scores

    # Identify indices of outliers (anomalies)
    anom_index = where(predictions == -1)
    anomalies = data_filtered.iloc[anom_index]
    anomalies['cluster'] = cluster_label  # Add the cluster column

    # Store anomalies in the dictionary
    out_anomalies_dict[cluster_label] = anomalies

    # Store anomaly scores for all samples in the dictionary
    data_filtered['cluster'] = cluster_label
    all_anomaly_dict[cluster_label] = data_filtered  # Add scores

    # Apply PCA to reduce spectral features to 2 principal components
    spectral_features = features.columns[1:]  # Exclude 'Clay_gkg'
    pca = PCA(n_components=2)
    data_pca = pca.fit_transform(data_filtered[spectral_features])

    # Create a DataFrame with PCs and the 'Clay_gkg' attribute
    data_3d = pd.DataFrame(data_pca, columns=['PC1', 'PC2'])
    data_3d['C_gkg'] = data_filtered['C_gkg'].values

    # Separate data into outliers and non-outliers
    outliers_3d = data_3d.iloc[anom_index]
    non_outliers_3d = data_3d.drop(index=anom_index[0])

    # Store data for plots in the dictionary
    plot_data_dict[cluster_label] = {
        'data_3d': data_3d,
        'outliers_3d': outliers_3d,
        'non_outliers_3d': non_outliers_3d,
        'pca': pca
    }

In [ ]:
### 3D plot Visualization ###

# Define the color palette for the clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3","#FF760A","#FFD700",
           "#0AB2D1", "#F781BF", "#999999", "#66C2A5"]

# Output directory for the plots
output_dir = "E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5/Carbon"

# Iterate over each cluster and generate plots
for idx, (cluster_label, results) in enumerate(plot_data_dict.items()):
    data_3d = results['data_3d']
    outliers_3d = results['outliers_3d']
    non_outliers_3d = results['non_outliers_3d']
    pca = results['pca']

    # Create 3D plot
    fig = plt.figure(figsize=(18, 16))
    ax = fig.add_subplot(111, projection='3d')

    # Plot outliers
    ax.scatter(outliers_3d['PC1'], outliers_3d['PC2'], outliers_3d['C_gkg'],
               c='red', marker='x', s=110, label="Outliers")

    # Plot non-outliers with cluster-specific color
    cluster_color = palette[idx % len(palette)]
    ax.scatter(non_outliers_3d['PC1'], non_outliers_3d['PC2'], non_outliers_3d['C_gkg'],
               c=cluster_color, marker='o', alpha=1, edgecolors='black', s=60, label="Non-outliers")


    # Label axes with explained variance of PCs and Clay attribute, and increase font size
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0] * 100:.2f}%)', fontweight='bold', fontsize=35, labelpad=37)
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1] * 100:.2f}%)', fontweight='bold', fontsize=35, labelpad=45)
    ax.set_zlabel('C gkg', fontweight='bold', fontsize=35, labelpad=54)

    # Increase the font size of the tick labels
    ax.tick_params(axis='both', which='major', labelsize=33, pad=9)  # Adjusts x and y axis
    ax.tick_params(axis='z', which='major', labelsize=33, pad=24)     # Adjusts z axis

    # Set z-axis limits from 0 to 900
    ax.set_zlim(0, 250)

    # Adjust layout to prevent label cutoff
    plt.tight_layout()

    # Add legend and title
    ax.legend(fontsize=12)
    plt.title(f'3D PCA plot with Outliers for Cluster {cluster_label} Oceania\n(Organic Carbon gkg vs. PC1 & PC2)', fontweight='bold', fontsize=20, y=1.05)

    # Save the plot as an image with specified DPI
    output_path = f"{output_dir}cluster_{cluster_label}_outlier_detection.png"
    plt.savefig(output_path, dpi=1000)

    # Display the plot
    plt.show()

In [ ]:
# Concatenate all DataFrames in the dictionary into a single DataFrame
anomalies_combined = pd.concat(out_anomalies_dict.values())

# Reset the index of the combined DataFrame
anomalies_combined = anomalies_combined.reset_index()

print(anomalies_combined)

In [ ]:
# Generate a list of all sample IDs in anomalies_combined
IDs_to_filter = anomalies_combined['ID_Unique'].tolist()

# Print the list of sample IDs
print(IDs_to_filter)

# Summary of the number of samples per cluster
summary = anomalies_combined.groupby('cluster').size()

# Print the summary by cluster
print(summary)

In [ ]:
# Concatenate all DataFrames in the dictionary into a single DataFrame
all_anomaly = pd.concat(all_anomaly_dict.values())

# Reset the index of the combined DataFrame
all_anomaly = all_anomaly.reset_index()

# Add a new column "outliers" filled with False
all_anomaly.insert(1, "outliers", False)

# Update the values to True where ID_Unique is in the list IDs_to_filter
all_anomaly.loc[all_anomaly['ID_Unique'].isin(IDs_to_filter), 'outliers'] = True

print(all_anomaly)

# Setting the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5/Carbon")

# Save all_anomaly DataFrame as a CSV file
all_anomaly.to_csv("all_anomaly_score.csv", index=False)

In [ ]:
DB_filter_step_5 = Data.reset_index()

# Add a new column "outliers_step_5_" filled with False
DB_filter_step_5.insert(1, "outliers_step_5_C", False)

# Update the values to True where ID_Unique is in the list IDs_to_filter
DB_filter_step_5.loc[DB_filter_step_5['ID_Unique'].isin(IDs_to_filter), 'outliers_step_5_C'] = True

# Create a new DataFrame with only the rows where outliers_step_5_ is True
outliers_samples_step_5 = DB_filter_step_5[DB_filter_step_5['outliers_step_5_C'] == True].copy()

# Delete the samples where outliers_step_5_ is True
DB_filter_step_5 = DB_filter_step_5[DB_filter_step_5['outliers_step_5_C'] == False].copy()

print(outliers_samples_step_5)

In [ ]:
# Checking the number of samples in the initial dataframe of the attribute that was analyzed,
# outliers outliers_samples_step_5, and DB_filter_step_5 dataframe

# Print the number of samples in Observation_final
print("Number of samples in Observation_final:", Observation_final.shape[0])

# Print the number of samples in outliers_samples_step_5_
print("Number of samples in outliers_samples_step_5_C:", outliers_samples_step_5.shape[0])

# Print the number of samples in DB_filter_step_5_
print("Number of samples in DB_filter_step_5_C:", DB_filter_step_5.shape[0])

In [ ]:
# Save the DB_filter_step_5 and outliers_samples_step_5 DataFrame as a CSV file

# Setting the working directory
os.chdir("E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5/Carbon")

# Save the outliers_samples_3 DataFrame as a CSV file
outliers_samples_step_5.to_csv("outliers_samples_step_5_C.csv", index=False)

# Save the DB_filter_3 DataFrame as a CSV file
DB_filter_step_5.to_csv("DB_multi_filter_step_5_C_Oceania_12.csv", index=False)

In [ ]:
from plotnine import *

# Create the color palette based on the number of clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF760A","#FFD700",
           "#0AB2D1", "#F781BF", "#999999", "#66C2A5", "#8C564B","#1F78B4",
           "#B2DF8A", "#FB9A99", "#FDBF6F", "#CAB2D6", "#6A3D9A"]

# Set the desired plot size
plot_width = 10
plot_height = 9

# Create the plot using plotnine
box_plot_1 = (ggplot(Observation_final, aes(x='factor(cluster)', y='C_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 250]) +
            labs(title='Box Plot of Organic Carbon gkg for Each Cluster before filter',
                 x='Cluster',
                 y='C gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Display the plot
print(box_plot_1)

# Create the plot using plotnine
box_plot_2 = (ggplot(DB_filter_step_5, aes(x='factor(cluster)', y='C_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 250]) +
            labs(title='Box Plot of Organic Carbon gkg for Each Cluster after filter',
                 x='Cluster',
                 y='C gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Display the plot
print(box_plot_2)

dpi_value = 1000

# Setting the working directory
output_path = "E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5/Carbon/box_plot_1_Oceania_multi_Carbon_before_step_5.png"
box_plot_1.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

output_path = "E:/OneDrive/Documentos/Doutorado/protocol_database/continents_protocol_validation/multiespectral_data_approach/Oceania/Step_5/Carbon/box_plot_2_Oceania_multi_Carbon_after_step_5.png"
box_plot_2.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from plotnine import ggplot, aes, geom_boxplot, scale_fill_manual, scale_y_continuous, labs, theme_minimal, theme, element_text, element_rect

# Create the color palette based on the number of clusters
palette = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00",
           "#FFFF33", "#A65628", "#F781BF", "#999999", "#66C2A5", "#8C564B",
           "#1F78B4", "#B2DF8A", "#FB9A99", "#FDBF6F", "#CAB2D6", "#6A3D9A"]

# Set the desired plot size
plot_width = 10
plot_height = 9

# Create the plot using plotnine for before the filter
box_plot_1 = (ggplot(Observation_final, aes(x='factor(cluster)', y='C_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 200]) +
            labs(title='Box Plot of C gkg for Each Cluster before filter',
                 x='Cluster',
                 y='C_gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Create the plot using plotnine for after the filter
box_plot_2 = (ggplot(DB_filter_step_5_C, aes(x='factor(cluster)', y='Clay_gkg', fill='factor(cluster)')) +
            geom_boxplot(alpha=0.7) +
            scale_fill_manual(values=palette) +
            scale_y_continuous(limits=[0, 200]) +
            labs(title='Box Plot of C gkg for Each Cluster after filter',
                 x='Cluster',
                 y='Clay_gkg') +
            theme_minimal() +
            theme(figure_size=(plot_width, plot_height),
                  plot_title=element_text(size=13, face="bold"),
                  axis_title_x=element_text(size=10, face="bold"),
                  axis_title_y=element_text(size=10, face="bold"),
                  plot_background=element_rect(fill="white")))

# Save the plots with high resolution (1000 dpi)
box_plot_1.save("box_plot_1.png", dpi=1000)
box_plot_2.save("box_plot_2.png", dpi=1000)

# Load the saved images
img1 = mpimg.imread("box_plot_1.png")
img2 = mpimg.imread("box_plot_2.png")

# Set up the side-by-side display
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(plot_width*2, plot_height))

# Display the images side by side
axes[0].imshow(img1)
axes[0].axis('off')  # Remove the axes
axes[0].set_title('Box Plot of C gkg for Each Cluster before filter')

axes[1].imshow(img2)
axes[1].axis('off')  # Remove the axes
axes[1].set_title('Box Plot of C gkg for Each Cluster after filter')

# Save the combined figure with high resolution (1000 dpi)
plt.tight_layout()
plt.savefig('combined_box_plots.png', dpi=1000)
plt.show()


# **Step 6° - ABOPA Filter**

## Importing and manipulating data

In [ ]:
# Packages required
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance
from joblib import Parallel, delayed
from tabulate import tabulate

In [ ]:
# Working directory settings
os.chdir("D:/alunos/Bruno_B/final_application/multiespectral_data_approach/Step_5_SySI_data/Organic_Carbon")

# ===============================================================
# DATA READING AND PREPARATION
DB_filter_step_5 = pd.read_csv("DB_filter_step_5_C_3.csv", header=0, sep=",", decimal=".", na_values=["NA", ""])

DB = DB_filter_step_5.copy()

In [ ]:
print(DB)

In [ ]:
# Selecting specific columns where the spectrum begins and ends

# Specify the range of spectral columns
wavelength_min = "Blue_SySI_Sentinel" # adjust the min spectral wavelength
wavelength_max = "SWIR_2_10m_SySI_Sentinel" # adjust the max spectral wavelength

# Select the spectral columns
DB_spectra = DB.loc[:, wavelength_min:wavelength_max] # soil spectra (original)

print(DB_spectra)

In [ ]:
# Creating a new dataframe with specific columns and removing rows with NA values

DB1 = pd.concat([DB.iloc[:, [0, 9]],  # Select the column number where the dependent variable is

                DB_spectra.iloc[:, list(range(0, 10, 1))]], axis=1)

DB1 = DB1.dropna()

# Printing the dataframe structure
print(DB1.info())

print(DB1)

# Separation of predictors and response variable
X = DB1.iloc[:, 2:11].values  # Covariates (columns 3 to 12)
y = DB1['C_gkg'].values        # Response variable

## ***Random Forest algorithm:***

In [ ]:
# ===============================================================
# AUXILIARY FUNCTIONS
# ===============================================================

def train_fold_model(model, X_train, y_train, X_val, y_val):
    """
    Trains the model on a fold and calculates evaluation metrics.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    r2 = r2_score(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rpiq = (np.percentile(y_val, 75) - np.percentile(y_val, 25)) / rmse
    return {'R2': r2, 'RMSE': rmse, 'RPIQ': rpiq}, model.feature_importances_

def summarize_fold_metrics(fold_metrics):
    """
    Calculates the average metrics across folds.
    """
    fold_metrics_df = pd.DataFrame(fold_metrics)
    avg_metrics = {
        'Average R2': fold_metrics_df['R2'].mean(),
        'Average RMSE': fold_metrics_df['RMSE'].mean(),
        'Average RPIQ': fold_metrics_df['RPIQ'].mean()
    }
    return avg_metrics, fold_metrics_df

def calculate_feature_importances(importances, feature_names):
    """
    Calculates the mean feature importances.
    """
    mean_importances = np.mean(importances, axis=0)
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Mean Importance': mean_importances
    }).sort_values(by='Mean Importance', ascending=False)
    return importance_df

In [ ]:
# ===============================================================
# INITIAL SETTINGS
# ===============================================================
# Initializing lists to store results
final_metrics = []
final_importances = []
final_predictions = []

# Set up the Random Forest model
define_model = RandomForestRegressor(random_state=None, n_jobs=5)

# Define hyperparameter grid
param_grid = {
    'n_estimators': [30, 100, 200, 500],
    'max_features': [2, 6],  # Number of variables per split
    'min_samples_leaf': [3, 6, 9],  # Minimum population in the leaf
    'max_leaf_nodes': [500, 1000, 1500],  # Maximum number of leaves
    'bootstrap': [True]
}

# Set up cross-validation (10-fold)
kf = KFold(n_splits=10, shuffle=True, random_state=None)

In [ ]:
# ===============================================================
# MODEL TRAINING
# ===============================================================
for model_num in range(50):
    print(f"\nTraining Model {model_num + 1}...")

    # Configure Grid Search with cross-validation
    grid_search = GridSearchCV(
        estimator=define_model,
        param_grid=param_grid,
        cv=kf,
        scoring='neg_mean_squared_error',
        n_jobs=60,
        verbose=0
    )

    # Fit the model and search for the best hyperparameters
    grid_search.fit(X, y)
    best_params = grid_search.best_params_
    print(f"Best hyperparameters for Model {model_num + 1}: {best_params}")

    # Best model with the optimal hyperparameters
    best_model = grid_search.best_estimator_

    # Model evaluation per fold
    fold_metrics = []
    importances = []

    for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
        # Split data into training and validation
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Train and evaluate the model on the current fold
        metrics, feature_importances = train_fold_model(
            best_model, X_train, y_train, X_val, y_val
        )

        # Store fold results
        fold_metrics.append({'Fold': fold, **metrics})
        importances.append(feature_importances)

    # Summary of metrics by model
    avg_metrics, fold_metrics_df = summarize_fold_metrics(fold_metrics)
    avg_metrics['Model'] = model_num + 1
    print(f"\nFold Metrics for Model {model_num + 1}:")
    print(tabulate(fold_metrics_df, headers='keys', tablefmt='pretty'))

    print(f"\nAverage Metrics for Model {model_num + 1}:")
    print(tabulate([avg_metrics], headers='keys', tablefmt='pretty'))

    # Feature importance
    feature_names = DB1.columns[2:11]  # Names of the covariates
    importance_df = calculate_feature_importances(importances, feature_names)
    print(f"\nFeature Importance for Model {model_num + 1}:")
    print(tabulate(importance_df, headers='keys', tablefmt='pretty'))

    # Store results
    final_metrics.append(avg_metrics)
    final_importances.append(importance_df)

    # Train the final model with all data
    best_model.fit(X, y)
    final_predictions.append(best_model.predict(X))

In [ ]:
# ===============================================================
# SUMMARY OF COVARIATE IMPORTANCE IN EACH ITERATION
# ===============================================================

# Initializing the list to store feature importance DataFrames
dfs_Mean_feature_importance = []

# Assuming 'final_importances' contains the DataFrames of feature importances
for i, df_fi in enumerate(final_importances):
    renamed = df_fi.rename(columns={"Mean Importance": f"Mean_Importance_Model_{i+1}"})
    dfs_Mean_feature_importance.append(renamed)

# Consolidating feature importances into a single DataFrame
all_Mean_feature_importance = dfs_Mean_feature_importance[0]
for df_fi in dfs_Mean_feature_importance[1:]:
    all_Mean_feature_importance = pd.merge(
        all_Mean_feature_importance,
        df_fi,
        on="Feature",
        how="inner",
        suffixes=("", f"_Model_{i+1}")  # Adding suffixes to avoid column duplication
    )

# Displaying the final result
print(all_Mean_feature_importance)

In [ ]:
# ===============================================================
# FINAL SUMMARY OF RESULTS
# ===============================================================

# Create DataFrame with model metrics
print("\nSummary of Metrics for All Models:")
final_metrics_df = pd.DataFrame(final_metrics)
print(tabulate(final_metrics_df, headers='keys', tablefmt='pretty'))

# Create the final DataFrame with IDs and observed values
all_observed_predicted = pd.DataFrame({'ID_Unico': DB['ID_Unico'], 'atr_content': y})

# Add predictions from each model to the final DataFrame
for model_num, preds in enumerate(final_predictions, 1):
    all_observed_predicted[f'Prediction_Model_{model_num}'] = preds

# Calculate the mean of predictions
all_observed_predicted['mean_pred'] = all_observed_predicted.filter(like='Prediction_Model_').mean(axis=1)

# Calculate the standard deviation of predictions
all_observed_predicted['Std_Dev_Prediction'] = all_observed_predicted.filter(like='Prediction_Model_').std(axis=1)

# Display the final DataFrame with new columns
print("\nFinal DataFrame with Predictions, Mean, and Standard Deviation:")
print(all_observed_predicted)

In [ ]:
# Setting the working directory
os.chdir("D:/alunos/Bruno_B/final_application/multiespectral_data_approach/Step_6_SySI_data/Carbon")

# Save the final_metrics_df DataFrame as a CSV file
final_metrics_df.to_csv("acuracia_DB_protocol_multi_filtered_step_5_Carbon.csv", index=False)

# Save all_observed_predicted DataFrame as a CSV file
all_observed_predicted.to_csv("predictions_DB_protocol_multi_filtered_step_5_Carbon.csv", index=False)

# Save all_Mean_feature_importance DataFrame as a CSV file
all_Mean_feature_importance.to_csv("ImpCoV_DB_protocol_multi_filtered_step_5_Carbon.csv", index=False)

## Cutting line determination

In [ ]:
# Packages required
import os
import statsmodels.api as sm
import numpy as np
import pandas as pd
from plotnine import *
import scipy.stats as stats
from plotnine import ggplot

In [ ]:
# Set the working directory
os.chdir("D:/alunos/Bruno_B/final_application/multiespectral_data_approach/Step_6_SySI_data/Carbon")

# Read the CSV file
stat_p = pd.read_csv("predictions_DB_protocol_multi_filtered_step_5_Carbon.csv")

print(stat_p)

In [ ]:
### Cutting line determination ###

# Specify the X and Y columns
column_x = stat_p['atr_content']
column_y = stat_p['mean_pred']

# Create a linear regression model
X = sm.add_constant(column_x)  # Add the constant for the intercept term
model = sm.OLS(column_y, X).fit()

# Calculate predicted values
predictions = model.predict(X)

# Calculate the standard deviation of residuals
residuals = model.resid
std_residuals = np.std(residuals)

# Confidence level (e.g., 98%)
confidence_level = 0.99

# Quantile of the t distribution
quantile_t = stats.t.ppf((1 + confidence_level) / 2, df=len(residuals) - 2)

# Calculate upper and lower confidence limits
upper_limit_attribute = predictions + quantile_t * (std_residuals * 1.15)
lower_limit_attribute = predictions - quantile_t * (std_residuals * 1.15)

# Regression coefficients
intercept, coef_x = model.params

# Standard error of coefficients
std_error = model.bse

# T-value and p-value of coefficients
t_value = model.tvalues
p_value = model.pvalues

# Coefficient of determination (R²)
r_squared_simulated = model.rsquared

# Fit a linear regression model for the upper limit
model_upper_limit = sm.OLS(upper_limit_attribute, X).fit()

# Fit a linear regression model for the lower limit
model_lower_limit = sm.OLS(lower_limit_attribute, X).fit()

# Coefficients of intercept and slope for the upper limit
coef_intercept_upper, coef_slope_upper = model_upper_limit.params

# Coefficients of intercept and slope for the lower limit
coef_intercept_lower, coef_slope_lower = model_lower_limit.params

# Display coefficients
print("Coefficients for Upper Limit:")
print("Intercept (β0):", round(coef_intercept_upper, 4))
print("Slope (β1):", round(coef_slope_upper, 4))

print("\nCoefficients for Lower Limit:")
print("Intercept (β0):", round(coef_intercept_lower, 4))
print("Slope (β1):", round(coef_slope_lower, 4))

In [ ]:
### Observed clay vs. predicted clay scatterplot ###

# Desired width and height of the plot
plot_width = 9
plot_height = 6

scatter_plot = (
    ggplot(stat_p, aes(x='atr_content', y='mean_pred')) +
    theme_classic() +
    geom_point(shape='o', color= 'black', fill= '#8069C7', alpha=1, size=1.5, stroke=0.2) + # Data points with black contour +  # Data points
    geom_smooth(method='lm', se=False, color='#000000', size=1.2) +  # Trend line
    geom_line(aes(y=upper_limit_attribute), color='#E41A1C', size=1) +  # Upper confidence interval line
    geom_line(aes(y=lower_limit_attribute), color='#E41A1C', size=1) +  # Lower confidence interval line

    annotate('text', x=200, y=850, label="β upper limit: {:.3f}".format(round(coef_intercept_upper, 3)),
             family="Arial", size=9)+
    annotate('text', x=200, y=817, label="β lower limit: {:.3f}".format(round(coef_intercept_lower, 3)),
             family="Arial", size=9)+
    annotate('text', x=200, y=784, label="R² model: {:.3f}".format(round(r_squared_simulated, 3)),
             family="Arial", size=9)+
    annotate('text', x=200, y=751, label="α: {:.3f}".format(round(coef_x, 3)),
             family="Arial", size=9)+

    #labs(x="Observed clay content (g kg⁻¹)", y="Predicted clay content (g kg⁻¹)") +
    labs(x="Observed Clay content (g kg$^{-1}$)", y="Predicted Clay by SySI content (g kg$^{-1}$)") +
    expand_limits(x=0, y=0) +  # Force the plot to show the origin (0,0)
    scale_y_continuous(limits=[0, 1000]) +
    scale_x_continuous(limits=[0, 1000]) +
    theme(
        axis_title_x=element_text(margin={'t': 12}, family="Arial", size=14, face="bold"),
        axis_title_y=element_text(margin={'r': 12}, family="Arial", size=14, face="bold"),
        axis_text_y=element_text(family="Arial", size=12),
        axis_text_x=element_text(family="Arial", size=12)
    )
)

# Display the plot
print(scatter_plot)

dpi_value = 1000

# Setting the working directory
output_path = "D:/alunos/Bruno_B/final_application/multiespectral_data_approach/Step_6_SySI_data/Clay/scatter_plot_DB_protocol_Clay_multi.png"
scatter_plot.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

scatter_plot

In [ ]:
### Observed C vs. predicted C scatterplot ###

# Desired width and height of the plot
plot_width = 9
plot_height = 6

scatter_plot = (
    ggplot(stat_p, aes(x='atr_content', y='mean_pred')) +
    theme_classic() +
    geom_point(shape='o', color= 'black', fill= '#8069C7', alpha=1, size=1.5, stroke=0.2) + # Data points with black contour +  # Data points
    geom_smooth(method='lm', se=False, color='#000000', size=1.2) +  # Trend line
    geom_line(aes(y=upper_limit_attribute), color='#E41A1C', size=1) +  # Upper confidence interval line
    geom_line(aes(y=lower_limit_attribute), color='#E41A1C', size=1) +  # Lower confidence interval line

    annotate('text', x=70, y=230, label="β upper limit: {:.3f}".format(round(coef_intercept_upper, 3)),
             family="Arial", size=9)+
    annotate('text', x=70, y=220, label="β lower limit: {:.3f}".format(round(coef_intercept_lower, 3)),
             family="Arial", size=9)+
    annotate('text', x=70, y=210, label="R² model: {:.3f}".format(round(r_squared_simulated, 3)),
             family="Arial", size=9)+
    annotate('text', x=70, y=200, label="α: {:.3f}".format(round(coef_x, 3)),
             family="Arial", size=9)+

    #labs(x="Observed clay content (g kg⁻¹)", y="Predicted clay content (g kg⁻¹)") +
    labs(x="Observed C content (g kg$^{-1}$)", y="Predicted C by SySI content (g kg$^{-1}$)") +
    expand_limits(x=0, y=0) +  # Force the plot to show the origin (0,0)
    scale_y_continuous(limits=[0, 250]) +
    scale_x_continuous(limits=[0, 250]) +
    theme(
        axis_title_x=element_text(margin={'t': 12}, family="Arial", size=14, face="bold"),
        axis_title_y=element_text(margin={'r': 12}, family="Arial", size=14, face="bold"),
        axis_text_y=element_text(family="Arial", size=12),
        axis_text_x=element_text(family="Arial", size=12)
    )
)

# Display the plot
print(scatter_plot)

dpi_value = 1000

# Setting the working directory
output_path = "D:/alunos/Bruno_B/final_application/multiespectral_data_approach/Step_6_SySI_data/Carbon/scatter_plot_DB_protocol_Carbon_multi.png"
scatter_plot.save(output_path, width=plot_width, height=plot_height, dpi=dpi_value)

scatter_plot

In [ ]:
### After the agreement analysis, filtering the samples in the database ###

# Create a subset of samples that are ABOVE the upper_limit_attribute line
samples_upper_limit = stat_p[stat_p['mean_pred'] > upper_limit_attribute]

# Create a subset of samples that are BELOW the lower_limit_attribute line
samples_lower_limit = stat_p[stat_p['mean_pred'] < lower_limit_attribute]

# Concatenate the dataframes samples_upper_limit and samples_lower_limit
outliers_samples = pd.concat([samples_upper_limit, samples_lower_limit])

# Merge the DataFrames based on the ID_Unique column and keep only the columns from the DB DataFrame
outliers_samples_6 = pd.merge(outliers_samples[['ID_Unico']], DB, on='ID_Unico', how='inner')

IDs_to_filter = outliers_samples['ID_Unico'].tolist()

DB_filter_step_6 = DB_filter_step_5

# Add a new column "outliers_step_4" filled with False
DB_filter_step_6.insert(1, "outliers_step_6", False)

# Update the values to True where ID_Unique is in the list IDs_to_filter
DB_filter_step_6.loc[DB_filter_step_6['ID_Unico'].isin(IDs_to_filter), 'outliers_step_6'] = True

# Create a new DataFrame with only the rows where outliers_step_4 is True
outliers_samples_step_6 = DB_filter_step_6[DB_filter_step_6['outliers_step_6'] == True].copy()

# Remove the samples where outliers_step_6 is True
DB_filter_step_6 = DB_filter_step_6[DB_filter_step_6['outliers_step_6'] == False].copy()

print(outliers_samples_step_6)


In [ ]:
# Checking the number of samples in the initial dataframe of the attribute that was analyzed,
# outliers outliers_samples_step_6, and DB_filter_step_6 dataframe

# Print the number of samples in DB_filter_step_5
print("Number of samples in DB_filter_step_5:", DB_filter_step_5.shape[0])

# Print the number of samples in outliers_samples_step_6
print("Number of samples in outliers_samples_step_6:", outliers_samples_step_6.shape[0])

# Print the number of samples in DB_filter_step_6
print("Number of samples in DB_filter_step_6:", DB_filter_step_6.shape[0])

In [ ]:
# Save the DB_filter_step_6 and outliers_samples_step_6 DataFrame as a CSV file
# Setting the working directory
os.chdir("D:/alunos/Bruno_B/final_application/multiespectral_data_approach/Step_6_SySI_data/Carbon")

# Save the outliers_samples_6 DataFrame as a CSV file
outliers_samples_step_6.to_csv("outliers_samples_multispectraldata_step_6_Carbon.csv", index=False)

# Save the DB_filter_6 DataFrame as a CSV file
DB_filter_step_6.to_csv("DB_protocol_multi_filter_step_6_Carbon_1.15.csv", index=False)

# **Clear all variables**

In [ ]:
# Remove todas as variáveis do ambiente
%reset -f

# Remove todas as imports e módulos carregados
import gc
gc.collect()